# **HeartView Pipeline for Empatica E4**
## ECG Signal Quality Assessment

This notebook presents an example of the HeartView pipeline's functionalities on some sample ECG data.

### 1. Import Modules

In [28]:
from sys import path
path.append('../')
from heartview import heartview
from heartview.pipeline import PPG, SQA
import pandas as pd
import matplotlib.pyplot as plt

### 2. Load Data

In [29]:
e4_zip = 'sample_e4_data.zip'
e4_data = PPG.preprocess_e4(e4_zip)

e4_data.keys()

dict_keys(['bvp', 'hr', 'ibi', 'acc', 'fs', 'start time'])

### 3. Setup Data

Set the parameters of your data, such as the sampling rate `fs` (Hz) and segment size `seg_size` (seconds).

In [30]:
fs = 64
seg_size = 60

Segment your data into 60-second windows.

In [31]:
bvp = heartview.segment_data(e4_data['bvp'], fs, seg_size)
bvp

,Segment,Unix,Timestamp,BVP,Peak
0,1,1.495437e+09,2017-05-22 03:15:25.000000,-0.00,NaN
1,1,1.495437e+09,2017-05-22 03:15:25.015625,-0.00,NaN
2,1,1.495437e+09,2017-05-22 03:15:25.031250,-0.00,NaN
3,1,1.495437e+09,2017-05-22 03:15:25.046875,-0.00,NaN
4,1,1.495437e+09,2017-05-22 03:15:25.062500,-0.00,NaN
...,...,...,...,...,...
503938,132,1.495445e+09,2017-05-22 05:26:39.031250,96.51,NaN
503939,132,1.495445e+09,2017-05-22 05:26:39.046875,92.92,NaN
503940,132,1.495445e+09,2017-05-22 05:26:39.062500,85.61,NaN
503941,132,1.495445e+09,2017-05-22 05:26:39.078125,75.98,NaN


### 4. Assess Signal Quality

##### First, detect the number of peaks (i.e., heartbeats) in each segment of the signal. Save each peak occurrence in a `Peak` column in the data frame.

In [32]:
# `e4_data` = output from `PPG.preprocess_e4()`
ibi = e4_data['ibi']
e4_fs = e4_data['fs']
start_time = e4_data['start time']

e4_peaks = PPG.get_e4_peaks(ibi, e4_fs, start_time)
e4_peaks.head()

,Unix,Timestamp,Second,Peak,IBI,HR
0,1.495437e+09,2017-05-22 03:15:25.000000,1.0,NaN,NaN,NaN
1,1.495437e+09,2017-05-22 03:15:25.015625,1.0,NaN,NaN,NaN
2,1.495437e+09,2017-05-22 03:15:25.031250,1.0,NaN,NaN,NaN
3,1.495437e+09,2017-05-22 03:15:25.046875,1.0,NaN,NaN,NaN
4,1.495437e+09,2017-05-22 03:15:25.062500,1.0,NaN,NaN,NaN


We can quickly verify that peaks have been detected.

In [33]:
e4_peaks['Peak'].sum()

3457.0

##### Compute second-by-second mean heart rate (HR) and interbeat interval (IBI) values and label all invalid values.

In [34]:
interval_data = PPG.get_e4_interval_data(e4_peaks, seg_size)
interval_data

,Segment,Second,Timestamp,Mean HR,Mean IBI,# R Peaks
0,1.0,1.0,2017-05-22 03:15:25,NaN,NaN,0.0
1,1.0,2.0,2017-05-22 03:15:26,NaN,NaN,0.0
2,1.0,3.0,2017-05-22 03:15:27,NaN,NaN,0.0
3,1.0,4.0,2017-05-22 03:15:28,NaN,NaN,0.0
4,1.0,5.0,2017-05-22 03:15:29,NaN,NaN,0.0
...,...,...,...,...,...,...
7851,131.0,7852.0,2017-05-22 05:26:16,65.081745,921.9175,2.0
7852,131.0,7853.0,2017-05-22 05:26:17,63.997065,937.5430,2.0
7853,131.0,7854.0,2017-05-22 05:26:18,59.997270,1000.0455,2.0
7854,131.0,7855.0,2017-05-22 05:26:19,59.074247,1015.6710,1.0


##### Compute the SQA metrics by segment.

In [35]:
peaks_by_seg = SQA.evaluate_peaks(interval_data, seg_size)
peaks_by_seg.head()

,Segment,Timestamp,Detected,Expected
0,1,2017-05-22 03:15:25,24.0,76.679121
1,2,2017-05-22 03:16:25,0.0,NaN
2,3,2017-05-22 03:17:25,22.0,83.200181
3,4,2017-05-22 03:18:25,117.0,77.311916
4,5,2017-05-22 03:19:25,95.0,75.786027


In [36]:
sqa = SQA.compute_metrics(peaks_by_seg)
sqa

,Segment,Timestamp,Detected,Expected,Invalid,Missing,% Missing
0,1,2017-05-22 03:15:25,24.0,76.679121,1,52.679121,68.700737
1,2,2017-05-22 03:16:25,0.0,NaN,1,0.000000,0.000000
2,3,2017-05-22 03:17:25,22.0,83.200181,1,61.200181,73.557750
3,4,2017-05-22 03:18:25,117.0,77.311916,0,0.000000,0.000000
4,5,2017-05-22 03:19:25,95.0,75.786027,0,0.000000,0.000000
...,...,...,...,...,...,...,...
126,127,2017-05-22 05:21:25,16.0,63.997065,1,47.997065,74.998853
127,128,2017-05-22 05:22:25,0.0,NaN,1,0.000000,0.000000
128,129,2017-05-22 05:23:25,75.0,60.709631,0,0.000000,0.000000
129,130,2017-05-22 05:24:25,83.0,61.932671,0,0.000000,0.000000


## Visualize Data

##### Visualize the raw and filtered ECG signal during Segment 1.

In [37]:
fs = 64
seg_size = 60
bvp = heartview.segment_data(e4_data['bvp'], fs, seg_size)
heartview.plot_signal(bvp, 'Timestamp', 'BVP', fs, seg_size, segment = 100, signal_type = 'bvp', peaks='Peak')

##### Visualize the expected-to-missing numbers of beats per segment.

In [38]:
SQA.plot_expected2missing(sqa, title = 'Sample PPG')